#MOUNT DRIVE



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#IMPORT LIBRARIES

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

# import torchvision.transforms as transforms
import torchvision
from torchvision import transforms, datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import copy
import random
import time
from tqdm import tqdm

#SELECT CUDA / CPU

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:",device)

Device: cuda


#SEED

In [ ]:
SEED = 100

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
random.seed(SEED)
np.random.seed(SEED)

#DATA

1. Choose Normalization & Augmentation

In [ ]:
AUGMENTATION = False
NORMALIZATION = True

if AUGMENTATION :
  print("AUGMENTATION")
  train_transform = transforms.Compose([
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomVerticalFlip(),
                              transforms.ColorJitter(brightness=2),
                              transforms.Resize((32,32)),
                              transforms.ToTensor()
                                        ])

  test_transform = transforms.Compose([
                            transforms.Resize((32,32)),
                             transforms.ToTensor()
                                       ])
  
elif AUGMENTATION and NORMALIZATION:
  print("AUGMENTATION and NORMALIZATION")
  train_transform = transforms.Compose([
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomVerticalFlip(),
                              transforms.ColorJitter(brightness=2),
                              transforms.Resize((32,32)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                        ])

  test_transform = transforms.Compose([
                             transforms.Resize((32,32)),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ])
  
elif NORMALIZATION and (not AUGMENTATION):
  print("No AUGMENTATION and NORMALIZATION")
  train_transform = transforms.Compose([
       transforms.Resize((32,32)),
       transforms.ToTensor(),
       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
       ])

  test_transform = transforms.Compose([
       transforms.Resize((32,32)),
       transforms.ToTensor(),
       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

elif (not NORMALIZATION) and (not AUGMENTATION):
  print("No AUGMENTATION and  No NORMALIZATION")
  train_transform = transforms.Compose(
      [transforms.Resize((32,32)),transforms.ToTensor()])

  test_transform = transforms.Compose(
      [transforms.Resize((32,32)),transforms.ToTensor()])



No AUGMENTATION and NORMALIZATION


2. Load Dataset

In [ ]:
CIFAR = False

if CIFAR:
  trainset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/CV-P3/CIFAR-data', train=True,
                                          download=True, transform=train_transform)

  testset = torchvision.datasets.CIFAR10(root='/content/drive/My Drive/CV-P3/CIFAR-data', train=False,
                                        download=True, transform=test_transform)
  
  print("Type - CIFAR")
else:

  TRAIN_PATH = '/content/drive/My Drive/CV-P3/TRAIN'
  TEST_PATH = '/content/drive/My Drive/CV-P3/TEST'

  trainset = datasets.ImageFolder(root= TRAIN_PATH, transform=train_transform)
  testset = datasets.ImageFolder(root= TEST_PATH, transform=test_transform)

  print("Type - FDDB")


print("Length of Trainset - ",len(trainset))
print("Length of Testset - ",len(testset))

Type - FDDB
Length of Trainset -  1999
Length of Testset -  200


3. Split dataset & get iterator object

In [ ]:
VAL_SPLIT = 0.9 # 90%- Train, 10%- Validation

train_examples = int(len(trainset) * VAL_SPLIT)
valid_examples = len(trainset) - train_examples



train_data_, valid_data_ = data.random_split(trainset, 
                                          [train_examples,valid_examples])



print(f'Complete data')
print(f'Images in training data: {len(train_data_)}')
print(f'Images in validation data: {len(valid_data_)}')
print(f'Images in test data: {len(testset)}')


valid_data_ = copy.deepcopy(valid_data_) # create a deep copy to avoid changing main object
valid_data_.dataset.transform = test_transform # apply test transforms to have consistency with test data


BATCH_SIZE = 32 # data batch_size

train_data = data.DataLoader(train_data_, 
                                shuffle = True, 
                                batch_size = BATCH_SIZE)

valid_data = data.DataLoader(valid_data_, 
                                batch_size = BATCH_SIZE)

test_data = data.DataLoader(testset, 
                                batch_size = BATCH_SIZE)

print(f'Batch Size: {BATCH_SIZE}')
print(f'Batches in training data: {len(train_data)}')
print(f'Batches in validation data: {len(valid_data)}')
print(f'Batches in test data: {len(test_data)}')


Complete data
Images in training data: 1799
Images in validation data: 200
Images in test data: 200
Batch Size: 32
Batches in training data: 57
Batches in validation data: 7
Batches in test data: 7


#CREATE MODEL - LeNet CNN

In [ ]:
class LeNet_V(nn.Module):
    def __init__(self, no_output):
        super().__init__()

        #first conv2d layer - [3,6,5]
        self.conv1 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
        
        nn.init.xavier_uniform_(self.conv1.weight)
        self.conv1_bn = nn.BatchNorm2d(6)
    
        #second conv2d layer - [6,16,5]
        self.conv2 = nn.Conv2d(in_channels = 6, 
                               out_channels = 16, 
                               kernel_size = 5)
        
        nn.init.xavier_uniform_(self.conv2.weight)
        self.conv2_bn = nn.BatchNorm2d(16)
        
        #fully connected layers
        self.fc_1 = nn.Linear(16 * 5 * 5, 120)
        nn.init.xavier_uniform_(self.fc_1.weight)
        self.fc_1_bn = nn.BatchNorm1d(120)
        
        self.fc_2 = nn.Linear(120, 84)
        nn.init.xavier_uniform_(self.fc_2.weight)
        self.fc_2_bn = nn.BatchNorm1d(84)
       
        self.fc_3 = nn.Linear(84, no_output)
        nn.init.xavier_uniform_(self.fc_3.weight)

    # Forward pass
    def forward(self, x):

        #x = [batch size, 3, 32, 32]
        
        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), kernel_size = 2))
        
        #x = [batch size, 6, 14, 14]
        
        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), kernel_size = 2))
        
        #x = [batch size, 16, 5, 5]
        
        
        x = x.view(x.shape[0], -1)
        
        #x = [batch size, 16*4*4 = 256]
        
        x = F.relu(self.fc_1_bn(self.fc_1(x)))
        
        #x = [batch size, 120]

        x = F.relu(self.fc_2_bn(self.fc_2(x)))
        
        x = self.fc_3(x)

        #x = [batch size, output dim]

        return x

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
if CIFAR:
  NO_OUTPUT = 10
else:
  NO_OUTPUT = 2

model = LeNet_V(NO_OUTPUT)

In [ ]:
print(f'The model has {count_parameters(model):,} trainable parameters')
print(model)

The model has 61,778 trainable parameters
LeNet_V(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv1_bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_1): Linear(in_features=400, out_features=120, bias=True)
  (fc_1_bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_2_bn): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_3): Linear(in_features=84, out_features=2, bias=True)
)


#MODEL UTILITY FUNCTIONS

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, data_iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # model in train mode
    model.train()
    
    for (x, y) in data_iterator:
        
        # load variable in device memory
        x = x.to(device)
        y = y.to(device)
        
        # make gradient as zero
        optimizer.zero_grad()
        
        # predict output
        y_pred = model(x)
        
        # calculate loss
        loss = criterion(y_pred, y)

        # backward propagation
        loss.backward()
        
        # calculate accuracy
        acc = calculate_accuracy(y_pred, y)
        
        # take step as per learning rate & regularization
        optimizer.step()
        
        # calculate losses
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(data_iterator), epoch_acc / len(data_iterator)

In [ ]:
def evaluate(model, data_iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # model in evaluation mode
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in data_iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(data_iterator), epoch_acc / len(data_iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def get_loss_before_train(model, data_iterator, criterion, device):
    
    epoch_loss = 0
    
    for (x, y) in tqdm(data_iterator):
        
        x = x.to(device)
        y = y.to(device)
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        epoch_loss += loss.item()
        
    return epoch_loss/len(data_iterator)

In [ ]:
def model_loss(train_data_iterator, device,reg=0):
  NO_OUTPUT = 2
  model = LeNet_V(NO_OUTPUT)

  criterion = nn.CrossEntropyLoss()

  model = model.to(device)
  criterion = criterion.to(device)

  loss_ = get_loss_before_train(model, train_data_iterator, criterion, device)
  
  return loss_


In [ ]:
def plot_metrics(TRAIN_ACC,VAL_ACC,TRAIN_LOSS,VAL_LOSS):
  plt.plot(np.asarray(TRAIN_ACC),'r',label="Training")
  plt.plot(np.asarray(VAL_ACC),'b',label="Validation")
  plt.title("Training & Validation Accuracy")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.legend(loc='best')
  plt.show()

  plt.plot(np.asarray(TRAIN_LOSS),'r',label="Training")
  plt.plot(np.asarray(VAL_LOSS),'b',label="Validation")
  plt.title("Training & Validation Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend(loc='best')
  plt.show()

#PRINT LOSS

In [ ]:
loss_ = model_loss(train_data,device,reg=0)
print("Loss: ",loss_)

100%|██████████| 57/57 [00:02<00:00, 19.72it/s]

Loss:  0.8399172038362738


In [ ]:
loss_ = model_loss(train_data,device,reg=10**3)
print("Loss: ",loss_)

100%|██████████| 57/57 [00:03<00:00, 18.95it/s]

Loss:  0.8471407555697257


#TRAIN MODEL

In [ ]:
def TRAIN(train_data_iterator,valid_data_iterator,device,optimizer,no_epochs,LR=0,reg=0):
  NO_OUTPUT = 2
  model = LeNet_V(NO_OUTPUT)

  if optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=reg, momentum = 0.9)
  elif optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=reg)

  criterion = nn.CrossEntropyLoss()

  model = model.to(device)
  criterion = criterion.to(device)

  EPOCHS = no_epochs

  best_valid_loss = float('inf')

  TRAIN_ACC = []
  VAL_ACC = []
  TRAIN_LOSS = []
  VAL_LOSS = []
  for epoch in tqdm(range(EPOCHS)):
      
      start_time = time.time()
      
      train_loss, train_acc = train(model, train_data_iterator, optimizer, criterion, device)
      valid_loss, valid_acc = evaluate(model, valid_data_iterator, criterion, device)
      
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'cv-babysitting-model.pt')
      
      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      TRAIN_ACC.append(train_acc)
      TRAIN_LOSS.append(train_loss)
      VAL_ACC.append(valid_acc)
      VAL_LOSS.append(valid_loss)
      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
def FINE_TRAIN(train_data_iterator,valid_data_iterator,device,optimizer,no_epochs,LR=0,reg=0):
  NO_OUTPUT = 2
  model = LeNet_V(NO_OUTPUT)

  if optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=reg, momentum = 0.9)
  elif optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=reg)

  criterion = nn.CrossEntropyLoss()

  model = model.to(device)
  criterion = criterion.to(device)

  EPOCHS = no_epochs

  best_valid_loss = float('inf')

  TRAIN_ACC = []
  VAL_ACC = []
  TRAIN_LOSS = []
  VAL_LOSS = []
  for epoch in tqdm(range(EPOCHS)):
      
      start_time = time.time()
      
      train_loss, train_acc = train(model, train_data_iterator, optimizer, criterion, device)
      valid_loss, valid_acc = evaluate(model, valid_data_iterator, criterion, device)
      
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'cv-babysitting-model.pt')
      
      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      TRAIN_ACC.append(train_acc)
      TRAIN_LOSS.append(train_loss)
      VAL_ACC.append(valid_acc)
      VAL_LOSS.append(valid_loss)
      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
      print(f'LR: {LR} | Reg: {reg}')

#1-
Overfit Model - Here we are overfitting the model. Initially a tiny batch of training was chosen for training so that model overfits. However, while using this tiny batch model overfits for that data in 2 epochs. In order to have large number of epochs (like 100-200) so that model overfits, entire training data of 1799 images was used for overfiting the model. It can be seen that model overfits.

Follwing are the learning rate(LR) and regularization (reg) used for training -

LR : 1e-3, reg : 0

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=200,LR=10**-3,reg=0)

  0%|          | 1/200 [00:07<25:37,  7.73s/it]

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.345 | Train Acc: 84.17%
	 Val. Loss: 0.179 |  Val. Acc: 92.86%


  1%|          | 2/200 [00:11<21:06,  6.40s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.161 | Train Acc: 93.91%
	 Val. Loss: 0.137 |  Val. Acc: 93.75%


  2%|▏         | 3/200 [00:14<17:54,  5.45s/it]

...

Epoch: 198 | Epoch Time: 0m 2s
	Train Loss: 0.002 | Train Acc: 100.00%
	 Val. Loss: 0.031 |  Val. Acc: 98.66%


100%|█████████▉| 199/200 [10:32<00:03,  3.05s/it]

Epoch: 199 | Epoch Time: 0m 2s
	Train Loss: 0.003 | Train Acc: 99.95%
	 Val. Loss: 0.029 |  Val. Acc: 98.21%


100%|██████████| 200/200 [10:35<00:00,  3.18s/it]

Epoch: 200 | Epoch Time: 0m 2s
	Train Loss: 0.003 | Train Acc: 99.95%
	 Val. Loss: 0.028 |  Val. Acc: 98.21%


#2-
In this step, we start tunning for learning rate selection. We will use brute force approach initially. We start with small regularization of 1e-6 (reg), which is kept constant throughout few of the next trials to verify the effect of learning rate.

Following are the learning rate (LR) & regularization(reg) -

reg : 1e-6, LR : 1e-6

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-6,reg=10**-6)

 10%|█         | 1/10 [00:03<00:28,  3.16s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.857 | Train Acc: 48.89%
	 Val. Loss: 0.792 |  Val. Acc: 52.68%


 20%|██        | 2/10 [00:06<00:25,  3.15s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.853 | Train Acc: 49.41%
	 Val. Loss: 0.791 |  Val. Acc: 49.11%


 30%|███       | 3/10 [00:09<00:21,  3.14s/it]

...

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.811 | Train Acc: 52.90%
	 Val. Loss: 0.760 |  Val. Acc: 58.04%


 90%|█████████ | 9/10 [00:28<00:03,  3.22s/it]

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.806 | Train Acc: 53.63%
	 Val. Loss: 0.748 |  Val. Acc: 58.93%


100%|██████████| 10/10 [00:31<00:00,  3.18s/it]

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: 0.802 | Train Acc: 53.88%
	 Val. Loss: 0.769 |  Val. Acc: 56.25%


As per above cell, it can be seen that losses i.e. train loss & validation loss barely change. They are more or less in the same range. This is mainly because for very low learning rate & losses never go down.

#3-
Here we keep same regularization of 1e-6 (reg) as previous step, and learning rate is kept as high as 1e6

Following are the learning rate (LR) & regularization(reg) -

reg : 1e-6, LR : 1e6

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**6,reg=10**-6)

 10%|█         | 1/10 [00:03<00:29,  3.29s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 50.13%
	 Val. Loss: nan |  Val. Acc: 54.91%


 20%|██        | 2/10 [00:06<00:26,  3.30s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 50.05%
	 Val. Loss: nan |  Val. Acc: 54.91%


 30%|███       | 3/10 [00:09<00:23,  3.31s/it]

...

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.26%
	 Val. Loss: nan |  Val. Acc: 54.91%


 90%|█████████ | 9/10 [00:29<00:03,  3.24s/it]

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.66%
	 Val. Loss: nan |  Val. Acc: 54.91%


100%|██████████| 10/10 [00:32<00:00,  3.26s/it]

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.66%
	 Val. Loss: nan |  Val. Acc: 54.91%


As per above cell, it can be seen that losses i.e. train loss & validation loss are nan i.e. reaching infinity. It comes to notice that the loss in exploding. This is mainly because for very high learning rate.

#4-

reg : 1e-6, LR : 3*e-3

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=3*10**-3,reg=10**-6)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:03<00:27,  3.10s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.690 | Train Acc: 67.46%
	 Val. Loss: 0.683 |  Val. Acc: 80.36%




 20%|██        | 2/10 [00:06<00:24,  3.10s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.674 | Train Acc: 71.29%
	 Val. Loss: 0.644 |  Val. Acc: 70.98%




 30%|███       | 3/10 [00:09<00:21,  3.11s/it]

...

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.252 | Train Acc: 91.31%
	 Val. Loss: 0.178 |  Val. Acc: 93.75%




 90%|█████████ | 9/10 [00:27<00:03,  3.09s/it]

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.249 | Train Acc: 91.58%
	 Val. Loss: 0.161 |  Val. Acc: 94.20%




100%|██████████| 10/10 [00:31<00:00,  3.10s/it]

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: 0.225 | Train Acc: 92.46%
	 Val. Loss: 0.148 |  Val. Acc: 95.09%


#I -
LR : 1e-3, reg = 1e-3

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-3,reg=10**-3)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:03<00:27,  3.09s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 49.69%
	 Val. Loss: 0.691 |  Val. Acc: 54.91%




 20%|██        | 2/10 [00:06<00:24,  3.10s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.690 | Train Acc: 56.35%
	 Val. Loss: 0.687 |  Val. Acc: 63.39%




 30%|███       | 3/10 [00:09<00:21,  3.10s/it]

...

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.613 | Train Acc: 71.51%
	 Val. Loss: 0.578 |  Val. Acc: 80.36%




 90%|█████████ | 9/10 [00:27<00:03,  3.08s/it]

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.580 | Train Acc: 76.48%
	 Val. Loss: 0.540 |  Val. Acc: 83.04%




100%|██████████| 10/10 [00:30<00:00,  3.09s/it]

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: 0.542 | Train Acc: 79.24%
	 Val. Loss: 0.495 |  Val. Acc: 87.95%


#II-
LR: 1e-3, reg = 1e3

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-3,reg=10**3)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:03<00:28,  3.12s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 51.14%
	 Val. Loss: 0.693 |  Val. Acc: 54.91%




 20%|██        | 2/10 [00:06<00:24,  3.10s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 50.62%
	 Val. Loss: 0.693 |  Val. Acc: 54.91%




 30%|███       | 3/10 [00:09<00:21,  3.09s/it]

...

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 49.79%
	 Val. Loss: 0.693 |  Val. Acc: 54.91%




 90%|█████████ | 9/10 [00:27<00:03,  3.07s/it]

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 49.22%
	 Val. Loss: 0.693 |  Val. Acc: 54.91%




100%|██████████| 10/10 [00:30<00:00,  3.08s/it]

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 51.14%
	 Val. Loss: 0.693 |  Val. Acc: 45.09%


#III-
LR: 1e-3, reg = 1e4

In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=10,LR=10**-3,reg=10**4)



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:03<00:27,  3.04s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.71%
	 Val. Loss: nan |  Val. Acc: 54.91%




 20%|██        | 2/10 [00:06<00:24,  3.06s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.82%
	 Val. Loss: nan |  Val. Acc: 54.91%




 30%|███       | 3/10 [00:09<00:21,  3.09s/it]

...

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.82%
	 Val. Loss: nan |  Val. Acc: 54.91%




 90%|█████████ | 9/10 [00:29<00:03,  3.19s/it]

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.82%
	 Val. Loss: nan |  Val. Acc: 54.91%




100%|██████████| 10/10 [00:32<00:00,  3.23s/it]

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 50.02%
	 Val. Loss: nan |  Val. Acc: 54.91%


#H1 - COARSE SEARCH

In [ ]:
max_count = 20

for count in tqdm(range(max_count)):
  LR = 10**random.uniform(-5,5)
  reg = 10**random.uniform(-3,-6)
  FINE_TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=5,LR=LR,reg=reg)
  print("-----------------------------------------------------------")




  0%|          | 0/20 [00:00<?, ?it/s]


  0%|          | 0/5 [00:00<?, ?it/s]


 20%|██        | 1/5 [00:03<00:12,  3.11s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 370354101136363905613824.000 | Train Acc: 47.66%
	 Val. Loss: 13.732 |  Val. Acc: 45.09%
LR: 18.33227620358218 | Reg: 1.2884057338060482e-06





 40%|████      | 2/5 [00:06<00:09,  3.10s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 6.403 | Train Acc: 48.58%
	 Val. Loss: 9.325 |  Val. Acc: 54.91%
LR: 18.33227620358218 | Reg: 1.2884057338060482e-06





 60%|██████    | 3/5 [00:09<00:06,  3.12s/it]

...

Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 50.02%
	 Val. Loss: nan |  Val. Acc: 54.91%
LR: 32.73914448848361 | Reg: 0.00017474358920171242





 80%|████████  | 4/5 [00:12<00:03,  3.09s/it]

Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 50.02%
	 Val. Loss: nan |  Val. Acc: 54.91%
LR: 32.73914448848361 | Reg: 0.00017474358920171242





100%|██████████| 5/5 [00:15<00:00,  3.08s/it]


100%|██████████| 20/20 [05:11<00:00, 15.59s/it]

Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: nan | Train Acc: 49.82%
	 Val. Loss: nan |  Val. Acc: 54.91%
LR: 32.73914448848361 | Reg: 0.00017474358920171242
-----------------------------------------------------------


#H2 - FINE SEARCH

In [ ]:
max_count = 10

for count in tqdm(range(max_count)):
  LR = 10**random.uniform(-4,-2)
  reg = 10**random.uniform(-3,-5)
  FINE_TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=5,LR=LR,reg=reg)
  print("-----------------------------------------------------------")



  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/5 [00:00<?, ?it/s]


 20%|██        | 1/5 [00:03<00:12,  3.11s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.695 | Train Acc: 50.21%
	 Val. Loss: 0.691 |  Val. Acc: 54.91%
LR: 0.0031441132469168112 | Reg: 7.304208193953323e-05





 40%|████      | 2/5 [00:06<00:09,  3.11s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 50.60%
	 Val. Loss: 0.692 |  Val. Acc: 57.14%
LR: 0.0031441132469168112 | Reg: 7.304208193953323e-05





 60%|██████    | 3/5 [00:09<00:06,  3.09s/it]

...

Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.694 | Train Acc: 49.82%
	 Val. Loss: 0.688 |  Val. Acc: 54.91%
LR: 0.0002100168339645272 | Reg: 2.526342788270967e-05





 80%|████████  | 4/5 [00:12<00:03,  3.08s/it]

Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.694 | Train Acc: 49.82%
	 Val. Loss: 0.688 |  Val. Acc: 54.91%
LR: 0.0002100168339645272 | Reg: 2.526342788270967e-05





100%|██████████| 5/5 [00:15<00:00,  3.09s/it]


100%|██████████| 10/10 [02:35<00:00, 15.52s/it]

Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 49.62%
	 Val. Loss: 0.688 |  Val. Acc: 54.91%
LR: 0.0002100168339645272 | Reg: 2.526342788270967e-05
-----------------------------------------------------------


#FINAL TRAINING

In [ ]:
LR = 0.007461627791541241
reg = 0.0001580451045434134
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=20,LR=LR,reg=reg)



  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:03<00:59,  3.11s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.694 | Train Acc: 49.82%
	 Val. Loss: 0.691 |  Val. Acc: 54.91%




 10%|█         | 2/20 [00:06<00:56,  3.11s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.693 | Train Acc: 50.13%
	 Val. Loss: 0.692 |  Val. Acc: 45.09%




 15%|█▌        | 3/20 [00:09<00:52,  3.10s/it]

...

Epoch: 18 | Epoch Time: 0m 3s
	Train Loss: 0.022 | Train Acc: 99.29%
	 Val. Loss: 0.078 |  Val. Acc: 97.77%




 95%|█████████▌| 19/20 [00:59<00:03,  3.11s/it]

Epoch: 19 | Epoch Time: 0m 3s
	Train Loss: 0.016 | Train Acc: 99.34%
	 Val. Loss: 0.054 |  Val. Acc: 99.11%




100%|██████████| 20/20 [01:02<00:00,  3.13s/it]

Epoch: 20 | Epoch Time: 0m 3s
	Train Loss: 0.013 | Train Acc: 99.45%
	 Val. Loss: 0.051 |  Val. Acc: 98.21%


#TEST MODEL

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

model.load_state_dict(torch.load('/content/cv-babysitting-model.pt'))
model = model.to(device)


In [ ]:
test_loss, test_acc = evaluate(model, test_data, criterion, device)
print(f'Test Accuracy: {test_acc} | Test Loss: {test_loss}')

Test Accuracy: 0.9955357142857143 | Test Loss: 0.01241226787013667


In [ ]:
LR = 0.007461627791541241
reg = 0.0001580451045434134
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=20,LR=LR,reg=reg)



  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:03<00:59,  3.13s/it]

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.243 | Train Acc: 89.11%
	 Val. Loss: 0.131 |  Val. Acc: 96.43%




 10%|█         | 2/20 [00:06<00:56,  3.12s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.107 | Train Acc: 96.49%
	 Val. Loss: 0.057 |  Val. Acc: 97.77%




 15%|█▌        | 3/20 [00:09<00:53,  3.12s/it]

...

Epoch: 18 | Epoch Time: 0m 3s
	Train Loss: 0.016 | Train Acc: 99.51%
	 Val. Loss: 0.023 |  Val. Acc: 98.66%




 95%|█████████▌| 19/20 [00:58<00:03,  3.04s/it]

Epoch: 19 | Epoch Time: 0m 3s
	Train Loss: 0.022 | Train Acc: 99.37%
	 Val. Loss: 0.041 |  Val. Acc: 98.66%




100%|██████████| 20/20 [01:01<00:00,  3.07s/it]

Epoch: 20 | Epoch Time: 0m 3s
	Train Loss: 0.027 | Train Acc: 99.22%
	 Val. Loss: 0.024 |  Val. Acc: 99.11%


In [ ]:
TRAIN(train_data,valid_data,device,optimizer='SGD',no_epochs=20,LR=10**-3,reg=10**-6)



  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:02<00:56,  2.98s/it]

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.693 | Train Acc: 50.38%
	 Val. Loss: 0.694 |  Val. Acc: 45.09%




 10%|█         | 2/20 [00:06<00:53,  3.00s/it]

Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.691 | Train Acc: 50.04%
	 Val. Loss: 0.691 |  Val. Acc: 44.64%




 15%|█▌        | 3/20 [00:09<00:51,  3.01s/it]

...

Epoch: 18 | Epoch Time: 0m 2s
	Train Loss: 0.253 | Train Acc: 91.85%
	 Val. Loss: 0.205 |  Val. Acc: 93.75%




 95%|█████████▌| 19/20 [00:57<00:02,  2.99s/it]

Epoch: 19 | Epoch Time: 0m 2s
	Train Loss: 0.230 | Train Acc: 92.87%
	 Val. Loss: 0.198 |  Val. Acc: 93.30%




100%|██████████| 20/20 [01:00<00:00,  3.02s/it]

Epoch: 20 | Epoch Time: 0m 2s
	Train Loss: 0.230 | Train Acc: 92.93%
	 Val. Loss: 0.203 |  Val. Acc: 91.52%
